In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import random
import math
from random import sample
import os 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from scipy.sparse import coo_matrix, csr_matrix, lil_matrix, vstack, hstack
from scipy import sparse

random.seed(10)

In [2]:
def read_txt(path):
    data_list = []
    with open(path) as f:
        for line in tqdm(f):
            line_data = []
            for word in line.split():
                line_data.append(float(word))
            data_list.append(line_data)
    return data_list

In [3]:
# import os


# path = 'C:/Users/bbb50/Desktop/Python/hw3/Yelp/'
# files = os.listdir(path)

# for filename in files:
#     portion = os.path.splitext(filename)
#     # 如果后缀是.dat
#     if portion[1] == ".dat":  
#         # 重新组合文件名和后缀名
#         newname = portion[0] + ".txt"   
#         os.rename(os.path.join(path, filename),os.path.join(path, newname))


In [8]:
user_business = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_business.txt')
user_business = pd.DataFrame(np.array(user_business).astype(int))
user_business.columns = ['user', 'business', 'rate']
# user_business = user_business.groupby("user").filter(lambda x: len(x) >= 3)
user_business.index = user_business.reset_index().index

user_label = preprocessing.LabelEncoder()
business_label = preprocessing.LabelEncoder()

user_business.user = user_label.fit_transform(user_business.user.values)
user_business.business = business_label.fit_transform(user_business.business.values)

user_list = user_business.user.value_counts().index.to_numpy()
user_list = np.sort(user_list)

business_list = user_business.business.value_counts().index.to_numpy()
business_list = np.sort(business_list)

198397it [00:00, 722206.02it/s]


In [5]:
user_compliment = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_compliment.txt')
user_compliment = pd.DataFrame(np.array(user_compliment)[:,:-1].astype(int))
user_compliment.columns = ['user', 'compliment']
user_compliment = user_compliment[user_compliment.user.isin(user_label.classes_)]

compliment_label = preprocessing.LabelEncoder()
user_compliment.compliment  = compliment_label.fit_transform(user_compliment.compliment.values)
user_compliment.user  = user_label.transform(user_compliment.user.values)

compliment_list = user_compliment.compliment.value_counts().index.to_numpy()
compliment_list = np.sort(compliment_list)

76875it [00:00, 571112.74it/s]


In [6]:
user_user = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_user.txt')
user_user = pd.DataFrame(np.array(user_user)[:,:-1].astype(int))
user_user.columns = ['user1', 'user2']
user_user = user_user[user_user.user1.isin(user_label.classes_) & user_user.user2.isin(user_label.classes_)]

user_user.user1 = user_label.transform(user_user.user1.values)
user_user.user2 = user_label.transform(user_user.user2.values)

158590it [00:00, 655341.28it/s]


In [7]:
business_city = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Yelp/business_city.txt')
business_city = pd.DataFrame(np.array(business_city)[:,:-1].astype(int))
business_city.columns = ['business', 'city']
business_city = business_city[business_city.business.isin(business_label.classes_)]

city_label = preprocessing.LabelEncoder()
business_city.city  = city_label.fit_transform(business_city.city.values)
business_city.business  = business_label.transform(business_city.business.values)

city_list = business_city.city.value_counts().index.to_numpy()
city_list = np.sort(city_list)


14267it [00:00, 837768.59it/s]


In [8]:
business_category = read_txt('C:/Users/bbb50/Desktop/Python/hw3/Yelp/business_category.txt')
business_category = pd.DataFrame(np.array(business_city).astype(int))
business_category.columns = ['business', 'category']
business_category = business_category[business_category.business.isin(business_label.classes_)]

category_label = preprocessing.LabelEncoder()
business_category.category  = category_label.fit_transform(business_category.category.values)
business_category.business  = business_label.transform(business_category.business.values)

category_list = business_category.category.value_counts().index.to_numpy()
category_list = np.sort(category_list)


40009it [00:00, 869022.12it/s]


In [9]:
user_business.reset_index().to_feather('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_business.ftr')
user_compliment.reset_index().to_feather('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_compliment.ftr')
user_user.reset_index().to_feather('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_user.ftr')
business_city.reset_index().to_feather('C:/Users/bbb50/Desktop/Python/hw3/Yelp/business_city.ftr')
business_category.reset_index().to_feather('C:/Users/bbb50/Desktop/Python/hw3/Yelp/business_category.ftr')



In [10]:
n_users = user_list.shape[0]
n_businesss = business_list.shape[0]
n_citys = city_list.shape[0]
n_categorys = category_list.shape[0]
n_compliments = compliment_list.shape[0]

In [11]:
b_ci_dict = {}
for b in tqdm(business_list):
    if b in business_city.business.values:
        b_ci = business_city[business_city.business == b].city.values[0]
        b_ci_dict[b] = b_ci
    else:
        b_ci_dict[b] = n_citys
        
city = []

for b in tqdm(user_business.business):
    city.append(b_ci_dict[b])

100%|█████████████████████████████████████████████████████████████████████| 188456/188456 [00:00<00:00, 1862523.07it/s]


In [12]:
b_ca_dict = {}
for b in tqdm(business_list):
    if b in business_category.business.values:
        b_ca = business_category[business_category.business == b].category.values[0]
        b_ca_dict[b] = b_ca
    else:
        b_ca_dict[b] = n_categorys
        
category = []

for b in tqdm(user_business.business):
    category.append(b_ca_dict[b])

100%|█████████████████████████████████████████████████████████████████████| 188456/188456 [00:00<00:00, 1384270.09it/s]


In [13]:
u_u_dict = {}
for u in tqdm(user_list):
    if u in user_user.user1.values:
        u_u = user_user[user_user.user1 == u].user2.values
        u_u_dict[u] = np.sort(u_u)
    else:
        u_u_dict[u] = [n_users]
user = []

for u in tqdm(user_business.user):
    user.append(u_u_dict[u])

100%|█████████████████████████████████████████████████████████████████████| 188456/188456 [00:00<00:00, 1532196.44it/s]


In [14]:
u_c_dict = {}
for u in tqdm(user_list):
    if u in user_compliment.user.values:
        u_c = user_compliment[user_compliment.user == u].compliment.values
        u_c_dict[u] = np.sort(u_c)
    else:
        u_c_dict[u] = [n_compliments]
        
compliment = []

for u in tqdm(user_business.user):
    compliment.append(u_c_dict[u])

100%|█████████████████████████████████████████████████████████████████████| 188456/188456 [00:00<00:00, 1811286.38it/s]


In [15]:
user_business['city'] = city
user_business['category'] = category
user_business['user_inter'] = user
user_business['compliment'] = compliment

In [16]:
user_business.reset_index().to_feather('C:/Users/bbb50/Desktop/Python/hw3/Yelp/user_business_fea.ftr')

In [17]:
user_business

,user,business,rate,city,category,user_inter,compliment
0,0,182,5,30,30,"[26, 99, 166, 182, 216, 227, 258, 263, 363, 46...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]"
1,0,200,5,30,30,"[26, 99, 166, 182, 216, 227, 258, 263, 363, 46...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]"
2,0,204,4,30,17,"[26, 99, 166, 182, 216, 227, 258, 263, 363, 46...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]"
3,0,453,3,30,34,"[26, 99, 166, 182, 216, 227, 258, 263, 363, 46...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]"
4,0,491,2,30,40,"[26, 99, 166, 182, 216, 227, 258, 263, 363, 46...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 10]"
...,...,...,...,...,...,...,...
188451,8531,13150,5,25,39,"[1906, 6381]","[6, 8, 10]"
188452,8532,2554,4,34,34,[8533],[11]
188453,8532,10435,1,34,28,[8533],[11]
188454,8532,12614,4,34,30,[8533],[11]


In [18]:
compliment_mat = lil_matrix((user_business.shape[0], (n_compliments+1)), dtype=int)
for ind in tqdm(user_business.index.to_numpy()):
    compliment_mat[ind, user_business.compliment[ind]] = 1
    
compliment_mat = compliment_mat.tocsr()

100%|███████████████████████████████████████████████████████████████████████| 188456/188456 [00:10<00:00, 17601.88it/s]


In [19]:
ui_mat = lil_matrix((user_business.shape[0], (n_users+1)), dtype=int)
for ind in tqdm(user_business.index.to_numpy()):
    ui_mat[ind, user_business.user_inter[ind]] = 1
    
ui_mat = ui_mat.tocsr()

100%|███████████████████████████████████████████████████████████████████████| 188456/188456 [00:11<00:00, 16028.80it/s]


In [20]:
row  = user_business.user.index.to_numpy()
data = np.ones(user_business.shape[0]).astype(int)

col  = user_business.user.values
user_mat = csr_matrix((data, (row, col)), shape=(user_business.shape[0], n_users))

col  = user_business.business.values
business_mat = csr_matrix((data, (row, col)), shape=(user_business.shape[0], n_businesss))

col  = user_business.city.values
city_mat = csr_matrix((data, (row, col)), shape=(user_business.shape[0], (n_citys+1)))

col  = user_business.category.values
category_mat = csr_matrix((data, (row, col)), shape=(user_business.shape[0], (n_categorys+1)))



In [21]:
ub_fea_mat = hstack([user_mat, business_mat, city_mat, category_mat, ui_mat, compliment_mat])
ub_fea_mat = ub_fea_mat.tocsr()
sparse.save_npz('C:/Users/bbb50/Desktop/Python/hw3/Yelp/ub_fea_mat.npz', ub_fea_mat, True) 

In [22]:
ub_fea_mat

<188456x31349 sparse matrix of type '<class 'numpy.intc'>'
	with 11640907 stored elements in Compressed Sparse Row format>

In [23]:
root = 'C:/Users/bbb50/Desktop/Python/hw3/Yelp'

train_val_list, test_list = [], []
test_ind_list = []

skf = StratifiedKFold(n_splits=5)
ub_i = user_business.index.to_numpy()

for train_index, test_index in skf.split(ub_i, user_business.user.to_numpy()):
    train, test =ub_i[train_index], ub_i[test_index]
    test_ind_list.append(test_index)
    train_val_list.append(user_business.loc[train])
    test_list.append(user_business.loc[test])
    
train_list, val_list = [], []
train_ind_list, val_ind_list = [], []

for df in train_val_list:
    train_ind, val_ind, _ , _ = train_test_split(df.index.to_numpy(), df.user.to_numpy(), stratify = df.user.to_numpy(), test_size = 1/8)
    
    val_list.append(df.loc[val_ind])
    train_list.append(df.loc[train_ind])    

    val_ind_list.append(val_ind)
    train_ind_list.append(train_ind)    
    

C:\Users\bbb50\anaconda3\envs\graph\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [24]:
sub_dir =  os.path.join(root, 'train')
if not(os.path.isdir(sub_dir)):
    os.makedirs(sub_dir)
for i, ind in enumerate(train_ind_list):
    
    path = os.path.join(sub_dir, 'train_ind'+str(i)+'.npy')
    np.save(path, ind)

sub_dir =  os.path.join(root, 'test')
if not(os.path.isdir(sub_dir)):
    os.makedirs(sub_dir)
for i, ind in enumerate(test_ind_list):
    
    path = os.path.join(sub_dir, 'test_ind'+str(i)+'.npy')
    np.save(path, ind)
    
sub_dir =  os.path.join(root, 'val')
if not(os.path.isdir(sub_dir)):
    os.makedirs(sub_dir)
for i, ind in enumerate(val_ind_list):
    
    path = os.path.join(sub_dir, 'val_ind'+str(i)+'.npy')
    np.save(path, ind)